# Instructions:
## Provide the path to the paragraph csv and question_answer csv under the variables para_data and question_ans.
## A finalprocessed.csv will be generated using which the required four csv's for evaluation will be generated in Final_data folder.
## Run all the cells as it is.
## The result will be saved at`{result_path}.csv`
## Scores will be printed in the last cell of evaluation section.


## File Generation and Preprocessing

In [ ]:
import os
import numpy as np

In [ ]:
'''
The 'para_data' and 'question_ans' variables take the files provided in the 
first stage of evaluation, i.e.,
'''
cwd=os.getcwd()

import pandas as pd
# Paragraph mapping file
para_data = pd.read_csv('https://drive.google.com/uc?export=download&id=1GNHhH81J1pEZSB-OSGRTtqIUQCCNhJ6I')
# Questions mapping file
question_ans = pd.read_csv('https://drive.google.com/uc?export=download&id=1GQ3-E7k60K16f47A18ptQfKUBkhxQudM')

In [ ]:
df = question_ans
df2 = para_data

In [ ]:
para_id_list = df.paragraph_id.tolist()
paras = []
for i in para_id_list:
    paras.append(df2['paragraph'][df2.id==i].tolist()[0])
df['paragraph'] = paras
df.rename(columns = {'question':'Question', 'theme':'Theme','answer':'Answer_text', 'paragraph':'Paragraph'}, inplace = True)
poss = []
for i in range(len(df)):
    poss.append('TRUE')
df['Answer_possible'] = poss
start = []
for i in range(len(df)):
  # print(df['Answer_text'][i])
  s = str(df['Answer_text'][i])
  start.append([df['Paragraph'][i].find(s)])
df['Answer_start'] = start
answers = df['Answer_text'].tolist()
ans = []
for i in answers:
    ans.append([i])
df['Answer_text'] = ans
df.to_csv(cwd+'/finaldataprocessed.csv')

In [ ]:
csv_save_path=cwd+'/finaldataprocessed.csv'

In [ ]:
df = pd.read_csv(csv_save_path)

In [ ]:
df = df.rename(columns={'Unnamed: 0':'id'})
df['id'] =df['id']+1
df.to_csv(cwd+'/finaldataprocessed.csv')

In [ ]:
os.mkdir(cwd+'/Final_data')

In [ ]:
m=cwd+'/Final_data'
df = pd.read_csv(cwd+'/finaldataprocessed.csv')

In [ ]:
paras = df.Paragraph.unique()

## Evaluation Files(4 CSVs) Generation

In [ ]:
val_folder=cwd+'/Final_data'
val_folder

In [ ]:
import os
if not os.path.exists(val_folder): os.makedirs(val_folder)

In [ ]:
df = pd.read_csv('finaldataprocessed.csv')

In [ ]:
paras = df.Paragraph.unique()

In [ ]:
df[df["Paragraph"]==paras[0]].Theme[0]

In [ ]:
data = []
for i, para in enumerate(paras):
    data_dict = {}
    data_dict['id']=i+1
    data_dict['paragraph']=para
    data_dict['theme'] = df[df["Paragraph"]==para].iloc[0].Theme
    data.append(data_dict)

In [ ]:
df2 = pd.DataFrame(data)

In [ ]:
df_theme=df2.copy()

In [ ]:
input_para=val_folder+'/input_para.csv'
df2.to_csv(input_para, header=True, index=False)

In [ ]:
data = []
for i in range(len(df.Question)):
    data_dict = dict()
    data_dict['id'] = i+1
    data_dict['question'] = df.Question[i]
    data_dict['theme'] = df.Theme[i]
    data.append(data_dict)

In [ ]:
df2 = pd.DataFrame(data)
input_question=val_folder+'/input_question.csv'
df2.to_csv(input_question, header=True, index=False)

In [ ]:
data = []
for i in range(len(df.Question)):
    data_dict = dict()
    data_dict['question_id'] = i+1
    if(df.Answer_start[i]=='[]'):
     data_dict['paragraph_id'] = -1
    else:
     data_dict['paragraph_id']=df_theme[df_theme.paragraph==df.Paragraph[i]].index.tolist()[0]+1
    data_dict['answers'] = df.Answer_text[i]
    data.append(data_dict)
df2 = pd.DataFrame(data)
ground_truth=val_folder+'/ground_truth.csv'
df2.to_csv(ground_truth, header=True, index=False)

In [ ]:
data = []
for i in df.Theme.unique():
    data_dict = dict()
    start, end = (df.index[df['Theme']==i][[0,-1]]+1).tolist()
    data_dict["theme"] = i
    data_dict["start"] = start
    data_dict['end'] = end
    data.append(data_dict)
df2 = pd.DataFrame(data)
theme_interval=val_folder+'/theme_interval.csv'
df2.to_csv(theme_interval, header=True, index=False)

In [ ]:
print(input_para+'\n'+input_question+'\n'+ground_truth+'\n'+theme_interval+'\n')

# Variables

In [ ]:


#Path to the folder containing evaluation dataset(4 csv's)
test_data_path=cwd+'/Final_data' 

#Path to store result file
result_path=cwd+'/result.csv'

#Name to store themewise file and delade predicted para file
m='pred_para'

#Provide paths to the evaluation files
theme_path = cwd+'/Final_data/theme_interval.csv'
ques_data_path = cwd+'/Final_data/input_question.csv'
para_data_path =cwd+'/Final_data/input_para.csv'

#Path to ground truth file
ground_truth_path = cwd+'/Final_data/ground_truth.csv'

# Imports

In [ ]:
!sudo apt install megatools

In [ ]:
#Variables not to be changed
seed = 42
threshold=0.1
similarity_method='dot'

In [ ]:
%cd $cwd

In [ ]:
!git clone https://github.com/castorini/dhr.git

In [ ]:
%cd dhr
!git clone https://huggingface.co/jacklin/DeLADE-CLS-P

In [ ]:
!megadl --path $cwd/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!luAz2IgI!q0R7urmIfhiTLhRpiBqs18Vpuriw6fVnQk_2ZkBPiBU'
!megadl --path $cwd/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!YmYBQZha!TjnxWDN0V_vp7-c3pxPLPmFPWW60UGy-yPuaaxrheD4'
!megadl --path $cwd/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!wz4VQYqR!oYV5DTqXhJ9BFwTeJE_BbeTnTlWQ8_JijUhgUhse9WE'
!megadl --path $cwd/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!8uIBhbgS!RUmCXrIYNfFeUahldqQ6jL2duHuTozJA6q_HpWi_SBU'

In [ ]:
# Allowed to make changes.
!pip install sentence-transformers
!pip install beir
!pip install datasets
!sudo apt install megatools

from sklearn.metrics.pairwise import cosine_similarity
import collections
import gdown
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
import pandas as pd
from google.colab import drive
from sentence_transformers import SentenceTransformer,util
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import time
import torch.nn.utils.prune as prune
import numpy as np
import os
from transformers import pipeline,set_seed
import torch
torch.manual_seed(42)
set_seed(42)


# Conversion to beir format

The evaluation data will be converted into beir format data.

In [ ]:
#Change the path to store beir format of evaluation data,if required
beir_data_path=cwd+'/beir_format' 

In [ ]:
import os
if not os.path.exists(beir_data_path): os.makedirs(beir_data_path)

In [ ]:
para_df=pd.read_csv(para_data_path)
ques_df=pd.read_csv(ques_data_path)
theme_df=pd.read_csv(theme_path)
ground_df=pd.read_csv(ground_truth_path)

In [ ]:
para_df.rename(columns={'theme':'title'},inplace=True)
para_df.rename(columns={'paragraph':'text'},inplace=True)
para_df.rename(columns={'id':'_id'},inplace=True)
para_df['_id'] = para_df['_id'].apply(lambda x: str(x))
para_df.to_csv(f'{beir_data_path}/corpus.csv',index=False)

In [ ]:
ques_df.rename(columns={'theme':'title'},inplace=True)
ques_df.rename(columns={'question':'text'},inplace=True)
ques_df.rename(columns={'id':'_id'},inplace=True)
ques_df['_id'] = ques_df['_id'].apply(lambda x: str(x))

In [ ]:
ground_df.pop('answers')

In [ ]:

ground_df['question_id'] = ground_df['question_id'].apply(lambda x: str(x))
ground_df['paragraph_id'] = ground_df['paragraph_id'].apply(lambda x: str(x))
ground_df.rename(columns={'paragraph_id':'corpus-id'},inplace=True)
ground_df.rename(columns={'question_id':'query-id'},inplace=True)
ground_df['score']=1

In [ ]:
import json
import os
themes=ques_df['title'].unique()
for theme in themes:
  os.mkdir(f'{beir_data_path}/{theme}')
  os.mkdir(f'{beir_data_path}/{theme}/qrels')
  corpus_df = para_df[para_df["title"]==theme]
  corpus_df.to_json(f'{beir_data_path}/{theme}/corpus.jsonl',orient='records',lines=True)
  queries_df=ques_df[ques_df["title"]==theme]
  queries_df.to_json(f'{beir_data_path}/{theme}/queries.jsonl',orient='records',lines=True)
  start = theme_df[theme_df['theme']==theme]['start'].tolist()[0]
  end = theme_df[theme_df['theme']==theme]['end'].tolist()[0]
  qrels=ground_df.iloc[start-1:end]
  qrels.to_csv(f'{beir_data_path}/{theme}/qrels/test.tsv',index=False,sep='\t')

# Delade generate embeddings


Run this cell to generate embeddings.


Embeddings will be saved at `/new_delade_embeddings_corpus.npy`

In [ ]:
%cd cwd+'/dhr'

In [ ]:
emb_delade_dhr_path=cwd+'/dhr'

In [ ]:
!pip install torch>=1.7.0
# !pip install transformers==4.15.0
!pip install pyserini
!pip install beir
!pip install datasets

In [ ]:
import pandas as pd
para_df=pd.read_csv(f'{beir_data_path}/corpus.csv')

In [ ]:
!$cwd/dhr/DeLADE-CLS-P

In [ ]:
#Will be generated in /cwd/new_delade_embeddings_corpus.npy
# Ignore the error. Some changes have been made for reducing the embedding generation time
themes=para_df['title'].unique()
for theme in themes:
  !python -m tevatron.datasets.beir.encode_and_retrieval_emb --theme f'{theme}' --model_name_or_path {cwd}/dhr/DeLADE-CLS-P --dataPath f'{beir_data_path}'

In [ ]:
embeddings_path=cwd+'/new_delade_embeddings_corpus.npy'

# Delade load embeddings

In [ ]:
import numpy as np
ab=np.load(embeddings_path,allow_pickle=True)
all_theme_embeddings=ab.item()

# Load QA base 

In [ ]:
%cd $cwd

In [ ]:
nlp=None

In [ ]:
model_name = "mrm8488/electra-small-finetuned-squadv2"

In [ ]:
nlp = pipeline("question-answering", model = model_name)

# Load Cross Encoder

In [ ]:
%cd $cwd

In [ ]:
!megadl --path {cwd}/ 'https://mega.nz/#!8zoUjZKL!iSOdg7ySmNut-mUaRDI0nO5uMZRJLg9R59DxLP3aQvU'

In [ ]:
!unzip {cwd}/minilml4-v2_17-20230205T094107Z-001.zip

In [ ]:
from sentence_transformers import CrossEncoder
cross_enc_ft_model=cwd+'/minilml4-v2_17'
cross_model = CrossEncoder(cross_enc_ft_model,max_length=512)


# Evaluation at k=3


In [ ]:
 %cd $cwd/dhr

In [ ]:
# Allowed to make changes.
def get_theme_model(theme):
  global_model = nlp
  return global_model
def Average(lis):
  return sum(lis)/len(lis)

In [ ]:

para_df2=pd.read_csv(f'{beir_data_path}/corpus.csv')
para_df=pd.read_csv(para_data_path)

In [ ]:
# Allowed to make changes.
#for direct delade(k=5)
def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]['theme']

  !python -m tevatron.datasets.beir.encode_and_retrieval_new --theme f'{theme}' --model_name_or_path {cwd}/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 

  for question in questions:
    ans = {}
    ans['question_id'] = question['id']    
    res5=pd.read_csv(cwd+f'/{m}_res.csv')
    l=len(para_df2.loc[para_df2['title']==theme].text.unique())
    
    if(l>=3):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].C.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]    
    elif(l==2):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]   
    elif(l==1):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]
    scores = cross_model.predict([(question['question'],p) for p in pred_para])
    best = np.argmax(scores)
    result = theme_model(question=question['question'], context=pred_para[best])
    answer = result['answer']
    if result['score'] < threshold:
      ans['paragraph_id'] = -1
      ans['answers'] = ''
    else:
      ans['answers'] = answer
      ans['paragraph_id'] = para_df[para_df['paragraph']==pred_para[best]]['id'].tolist()[0]
    pred_out.append(ans)

In [ ]:
# NOT allowed to make changes. 
from tqdm import tqdm
# All theme prediction.
questions = json.loads(pd.read_csv(ques_data_path).to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv(theme_path).to_json(orient="records"))
pred_out = []
theme_inf_time = {}
execution_times = []
res=pd.DataFrame()
res['id']=None
res['A']=None
res['B']=None
res['C']=None
res['D']=None
res['E']=None
res.to_csv(cwd+f'/{m}_res.csv',index=False)
for theme_interval in tqdm(theme_intervals):
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  execution_times.append(execution_time)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv(result_path, index=False)
theme_inf_df = pd.DataFrame(list(theme_inf_time.items()),columns = ['theme','avg_inf_time']) 
theme_inf_df.to_csv(cwd+f'/{m}_inf_time.csv', index=False)

In [ ]:
lst=theme_inf_df['avg_inf_time']
theme_int=pd.read_csv(theme_path)
theme_times = [ex_time/(theme_int['end'][i]-theme_int['start'][i]+1) for i,ex_time in enumerate(lst)]

In [ ]:

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(str(s)))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  try:
    ground_truths[0]
  except Exception as e:
    if predicted!=predicted:
      max_f1 = 1
  return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(str(s)))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  try:
    ground_truths[0]
  except Exception as e:
    if predicted!=predicted:
      max_f1 = 1
  return max_f1

In [ ]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
total_f1=0
total_f2=0
pred = pd.read_csv(result_path)
truth = pd.read_csv(ground_truth_path)
truth.paragraph_id = truth.paragraph_id
# truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
# truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv(ques_data_path)
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = truth_row["paragraph_id"]
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == -1:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
    total_f2+=1
  elif predicted_paragraph == truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
    total_f2+=1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1
  total_f1+=f1
final_f1 = round(total_f1/len(questions),3)
final_f2 = round(total_f2/len(questions),3)

In [ ]:
# NOT allowed to make changes.
theme_inf_df = pd.read_csv(cwd+f'/{m}_inf_time.csv')
theme_inf_time = {theme:theme_inf_df[theme_inf_df['theme']==theme]['avg_inf_time'].tolist()[0] for theme in metrics}
no_of_themes=len(theme_inf_df)
# Final score.
inf_time_threshold = 1000.0  # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0

for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += 1/(no_of_themes) * inf_time_score * qa_score
  final_para_score += 1/(no_of_themes) * inf_time_score * para_score

print('f1_score:',final_f1*100)
print('para_score:',round(final_para_score,3)*100)
print('qa_score:',round(final_qa_score,3)*100)
print('average_inference_time:',round(Average(theme_times),0))
print('median time:',round(np.sort(theme_times)[len(theme_times)//2],0))


# Evaluation at k=3(with caching)


In [ ]:
 %cd $cwd/dhr

In [ ]:
# Allowed to make changes.
def get_theme_model(theme):
  global_model = nlp
  return global_model
def Average(lis):
  return sum(lis)/len(lis)

In [ ]:

para_df2=pd.read_csv(f'{beir_data_path}/corpus.csv')
para_df=pd.read_csv(para_data_path)

In [ ]:
num_ques_cache = 0 # number of question cached

from sklearn.metrics.pairwise import cosine_similarity
# evaluation function
def pred_theme_ans(questions, theme_model, pred_out):

  theme = questions[0]['theme']


  !python -m tevatron.datasets.beir.encode_and_retrieval_new --theme f'{theme}' --model_name_or_path {cwd}/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 

  
  # a file 'delade_embeddings_queries.npy' containing embeddings of all the questions in the theme is created
  query_embeds = np.load(cwd+'/delade_embeddings_queries.npy',allow_pickle=True).item() # loading embeddings of all the queries in the theme
  cos_scores = cosine_similarity(query_embeds['queries']) # computing cosine similarity scores of all the possible pairs of queries in 'query_embeds'
  cached_paras = []

  for k,question in enumerate(questions):

    ans = {}
    ans['question_id'] = question['id'] 
    final_para = 'blank'

    if k>0:
      if np.max(cos_scores[k][:k]) > 0.8: 
        final_para = cached_paras[np.argmax(cos_scores[k][:k])] # use the cached para if similarity score is greater than 0.8
        result = theme_model(question=question['question'], context=final_para)
        cached_paras.append(final_para)  
        global num_ques_cache
        num_ques_cache += 1

    if final_para == 'blank':
      res5=pd.read_csv(cwd+f'/{m}_res.csv')
      l=len(para_df2.loc[para_df2['title']==theme].text.unique())
      if(l>=3):
        pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].C.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]    
      elif(l==2):
        pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]   
      elif(l==1):
        pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]
      scores = cross_model.predict([(question['question'],p) for p in pred_para])
      final_para = pred_para[np.argmax(scores)]
      result = theme_model(question=question['question'], context=final_para)
      cached_paras.append(final_para)

    answer = result['answer']
    if result['score'] < threshold:
      ans['paragraph_id'] = -1
      ans['answers'] = ''
    else:
      ans['answers'] = answer
      ans['paragraph_id'] = 1 + para_df.index[para_df['paragraph']==final_para].tolist()[0]
      
    pred_out.append(ans)

In [ ]:
# NOT allowed to make changes. 
from tqdm import tqdm
# All theme prediction.
questions = json.loads(pd.read_csv(ques_data_path).to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv(theme_path).to_json(orient="records"))
pred_out = []
theme_inf_time = {}
execution_times = []
res=pd.DataFrame()
res['id']=None
res['A']=None
res['B']=None
res['C']=None
res['D']=None
res['E']=None
res.to_csv(cwd+f'/{m}_res.csv',index=False)
for theme_interval in tqdm(theme_intervals):
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  execution_times.append(execution_time)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv(result_path, index=False)
theme_inf_df = pd.DataFrame(list(theme_inf_time.items()),columns = ['theme','avg_inf_time']) 
theme_inf_df.to_csv(cwd+f'/{m}_inf_time.csv', index=False)

In [ ]:
lst=theme_inf_df['avg_inf_time']
theme_int=pd.read_csv(theme_path)
theme_times = [ex_time/(theme_int['end'][i]-theme_int['start'][i]+1) for i,ex_time in enumerate(lst)]

In [ ]:

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(str(s)))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  try:
    ground_truths[0]
  except Exception as e:
    if predicted!=predicted:
      max_f1 = 1
  return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(str(s)))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  try:
    ground_truths[0]
  except Exception as e:
    if predicted!=predicted:
      max_f1 = 1
  return max_f1

In [ ]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
total_f1=0
total_f2=0
pred = pd.read_csv(result_path)
truth = pd.read_csv(ground_truth_path)
truth.paragraph_id = truth.paragraph_id
# truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
# truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv(ques_data_path)
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = truth_row["paragraph_id"]
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == -1:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
    total_f2+=1
  elif predicted_paragraph == truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
    total_f2+=1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1
  total_f1+=f1
final_f1 = round(total_f1/len(questions),3)
final_f2 = round(total_f2/len(questions),3)

In [ ]:
# NOT allowed to make changes.
theme_inf_df = pd.read_csv(cwd+f'/{m}_inf_time.csv')
theme_inf_time = {theme:theme_inf_df[theme_inf_df['theme']==theme]['avg_inf_time'].tolist()[0] for theme in metrics}
no_of_themes=len(theme_inf_df)
# Final score.
inf_time_threshold = 1000.0  # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0

for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += 1/(no_of_themes) * inf_time_score * qa_score
  final_para_score += 1/(no_of_themes) * inf_time_score * para_score

print('f1_score:',final_f1*100)
print('para_score:',round(final_para_score,3)*100)
print('qa_score:',round(final_qa_score,3)*100)
print('average_inference_time:',round(Average(theme_times),0))
print('median time:',round(np.sort(theme_times)[len(theme_times)//2],0))
print('percent questions cached:',round(num_ques_cache/len(questions),3)*100)